In [6]:
import torch.nn as nn
import torch
from torchsummary import summary

In [8]:
class NetG(nn.Module):
    """
    GENERATOR NETWORK
    """

    def __init__(self, opt):
        super(NetG, self).__init__()
        ngf = 64
        self.e1 = nn.Sequential(
            nn.Conv2d(opt.nc, ngf, 4, 2, 1, bias=False),            # 32*32
        )
        self.e2 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf, ngf << 1, 4, 2, 1, bias=False),          # 16*16
            nn.BatchNorm2d(ngf << 1),       # 128
        )
        self.e3 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf << 1, ngf << 2, 4, 2, 1, bias=False),     # 8*8
            nn.BatchNorm2d(ngf << 2),       # 256
        )
        self.e4 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf << 2, ngf << 3, 4, 2, 1, bias=False),     # 4*4
            nn.BatchNorm2d(ngf << 3),       # 512
        )
        self.e5 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf << 3, ngf << 3, 4, 2, 1, bias=False),     # 4*4
            nn.BatchNorm2d(ngf << 3),       # 512
        )
        self.e6 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf << 3, ngf << 3, 4, 2, 1, bias=False),     # 4*4
            nn.BatchNorm2d(ngf << 3),       # 512
        )
        self.e7 = nn.Sequential(
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf << 3, ngf << 3, 4, 2, 1, bias=False),     # 4*4
            nn.ReLU(True),      # 512
            nn.ConvTranspose2d(ngf << 3, ngf << 3, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf << 3),  
        )
        
        
        self.d6 = nn.Sequential(
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf << 4, ngf << 3, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf << 3),
        )
        self.d5 = nn.Sequential(
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf << 4, ngf << 3, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf << 3),
        )
        self.d4 = nn.Sequential(
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf << 4, ngf << 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf << 2),
        )
        self.d3 = nn.Sequential(
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf << 3, ngf << 1, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf << 1),
        )
        self.d2 = nn.Sequential(
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf << 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
        )
        self.d1 = nn.Sequential(
            nn.ConvTranspose2d(ngf<< 1, opt.nc, 4, 2, 1, bias=False),
            nn.Tanh(),
        )

    def forward(self, x):
        e1_out = self.e1(x)
        e2_out = self.e2(e1_out)
        e3_out = self.e3(e2_out)
        e4_out = self.e4(e3_out)
        e5_out = self.e5(e4_out)
        e6_out = self.e6(e5_out)
        e7_out = self.e7(e6_out)
        
        d6_out = self.d6(torch.cat([e7_out, e6_out], dim=1))
        d5_out = self.d5(torch.cat([e5_out, d6_out], dim=1))
        d4_out = self.d4(torch.cat([e4_out, d5_out], dim=1))
        d3_out = self.d3(torch.cat([e3_out, d4_out], dim=1))
        d2_out = self.d2(torch.cat([e2_out, d3_out], dim=1))
        d1_out = self.d1(torch.cat([e1_out, d2_out], dim=1))
        
        return d1_out

In [9]:
import time
model=NetG()
# start = time.time()
input=torch.rand((1,3,128,128))
# end = time.time()
out=model(input)
# print((end-start)*1000)

In [10]:
out.shape


torch.Size([1, 3, 128, 128])

In [18]:
class NetD(nn.Module):
    def __init__(self):
        super(NetD, self).__init__()
        ngf = 64
        self.model = nn.Sequential(
            nn.Conv2d(3, ngf, 4, 2, 1, bias=False),

            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf, ngf << 1, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf << 1),       # 128

            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf << 1, ngf << 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf << 2),       # 256

            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf << 2, ngf << 3, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf << 3),       # 512

            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf << 3, ngf << 4, 4, 2, 1, bias=False),       # 512
            nn.BatchNorm2d(ngf << 4),       # 100
            
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ngf << 4, 100, 4, 1, 0, bias=False),       # 512   # 100
        )

        self.classify = nn.Sequential(
            nn.Conv2d(100, 1, 3, 1, 1, bias=False),       # 512
            nn.Sigmoid(),
        )


    def forward(self, x):
        feature = self.model(x)
        classification = self.classify(feature)
        return classification.view(-1, 1).squeeze(1), feature


In [19]:
import time
model1=NetD()
# start = time.time()
input1=torch.rand((1,3,128,128))
# end = time.time()
out1,out2=model1(input1)
# print((end-start)*1000)

In [20]:
out1.shape,out2.shape

(torch.Size([1]), torch.Size([1, 100, 1, 1]))